In [ ]:
!pip install openai
!pip install langchain
!pip install dotenv

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.environ['API2D_API_KEY']

ModuleNotFoundError: No module named 'dotenv'